In [1]:
import os
import re
import random
import urllib.request
import numpy as np
np.random.seed(seed=123) # Do not change
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation

import re

In [2]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

### import text dataset

from google.colab import drive
drive.mount('/content/drive')

file_path1 = '/content/drive/MyDrive/trump_speeches/trump_sou_2018.txt'
file_path2 = '/content/drive/MyDrive/trump_speeches/trump_sou_2019.txt'
file_path3 = '/content/drive/MyDrive/trump_speeches/trump_sou_2020.txt'

with open(file_path1, encoding="latin-1") as file:
    content1 = file.read()

with open(file_path2, encoding="latin-1") as file:
    content2 = file.read()

with open(file_path3, encoding="latin-1") as file:
    content3 = file.read()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install hmmlearn

import re
import nltk
import numpy as np
from collections import defaultdict
from hmmlearn import hmm
from sklearn.preprocessing import LabelEncoder
from itertools import count

# Download resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk import pos_tag

import matplotlib.pyplot as plt
from wordcloud import WordCloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 3.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
# Sample text
text_trump = content1 + content2 + content3

In [5]:
# supervised

def supervisedHMM(text):
  text = re.sub(r'[\\$\\(\\)\\;\\:]','', text)

  # 1. Sentence Splitting
  sentences = re.split(r'[!?\.]\s*', text)
  sentences = [s for s in sentences if s]  # Remove empty strings

  # 2. Tokenization and POS Tagging
  tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]
  tokens = [[word.lower() for word in sentence if not word in [', ', ',']] for sentence in tokenized_sentences]
  pos_tags = [pos_tag(sentence) for sentence in tokens]

  word_to_num = {word: i for i, word in enumerate(sorted(set(word for sent in tokens for word in sent)))}
  num_to_word = {v: k for k, v in word_to_num.items()}  # Reverse mapping

  pos = [tag for sent in pos_tags for _, tag in sent]
  pos_to_num = {tag: i for i, tag in enumerate(sorted(set(pos)))}
  num_to_pos = {v: k for k, v in pos_to_num.items()}  # Reverse mapping

  # Convert sentences to numeric representation
  numeric_sentences = [[word_to_num[word] for word in sent] for sent in tokens]
  numeric_pos = [[pos_to_num[tag] for _, tag in sent] for sent in pos_tags]

  total_words = set([element for sentence in numeric_sentences for element in sentence])
  total_pos = set([element for sentence in numeric_pos for element in sentence])

  A = [[0 for _ in range(len(total_pos))] for _ in range(len(total_pos))]
  O = [[0 for _ in range(len(total_words))] for _ in range(len(total_pos))]

  for start_state in range(len(A)):
    for end_state in range(len(A[0])):
      count_num = 0
      count_denom = 0
      for sentence in numeric_pos:
        for i in range(len(sentence)-1):
          if sentence[i] == start_state and sentence[i+1] == end_state:
            count_num += 1
          if sentence[i] == start_state:
            count_denom += 1
      if count_denom != 0:
        A[start_state][end_state] = count_num / count_denom
      else:
        A[start_state][end_state] = 0


  for state in range(len(O)):
    print(state)
    for emission in range(len(O[0])):
      count_num = 0
      count_denom = 0
      for i in range(len(numeric_sentences)):
        for j in range(len(numeric_sentences[i])):
          if numeric_sentences[i][j] == emission and numeric_pos[i][j] == state:
            count_num += 1
          if numeric_pos[i][j] == state:
            count_denom += 1
      if count_denom != 0:
        O[state][emission] = count_num / count_denom
      else:
        O[state][emission] = 0

  return A, O, num_to_word, num_to_pos


In [6]:
def generate_top(A, O, num_to_word, num_to_pos):
  top_words_per_state = []

  for state_idx in range(len(O)):
      # Get the probabilities for the current state (row)
      state_probs = O[state_idx]

      # Get the indices of the top 10 words for the current state
      top_indices = np.argsort(state_probs)[-10:]  # Get the indices of the top 10 highest values

      # Get the corresponding words and probabilities
      top_words = [(num_to_word[word_idx], state_probs[word_idx]) for word_idx in top_indices if state_probs[word_idx] != 0]

      # Store the result for the current state
      top_words_per_state.append(top_words)


  # Print the top 10 words for each state
  for state_idx, top_words in enumerate(top_words_per_state):
      state = num_to_pos.get(state_idx)
      print(f"State {state}:")

      for word, prob in top_words:
        if prob != 0:
          print(f"  Word: '{word}', Probability: {prob:.4f}")
      print()  # Newline for better separation between states


  return top_words_per_state


In [7]:
A_trump, O_trump, num_to_word_trump, num_to_pos_trump = supervisedHMM(text_trump)

print(num_to_pos_trump)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
{0: '$', 1: 'CC', 2: 'CD', 3: 'DT', 4: 'EX', 5: 'FW', 6: 'IN', 7: 'JJ', 8: 'JJR', 9: 'JJS', 10: 'MD', 11: 'NN', 12: 'NNP', 13: 'NNPS', 14: 'NNS', 15: 'PDT', 16: 'PRP', 17: 'PRP$', 18: 'RB', 19: 'RBR', 20: 'RBS', 21: 'RP', 22: 'TO', 23: 'VB', 24: 'VBD', 25: 'VBG', 26: 'VBN', 27: 'VBP', 28: 'VBZ', 29: 'WDT', 30: 'WP', 31: 'WP$', 32: 'WRB'}


In [8]:
file_path1 = '/content/drive/MyDrive/obama_speeches/obama_sou_2010.txt'
file_path2 = '/content/drive/MyDrive/obama_speeches/obama_sou_2011.txt'
file_path3 = '/content/drive/MyDrive/obama_speeches/obama_sou_2012.txt'
file_path4 = '/content/drive/MyDrive/obama_speeches/obama_sou_2013.txt'
file_path5 = '/content/drive/MyDrive/obama_speeches/obama_sou_2014.txt'
file_path6 = '/content/drive/MyDrive/obama_speeches/obama_sou_2015.txt'
file_path7 = '/content/drive/MyDrive/obama_speeches/obama_sou_2016.txt'

with open(file_path1, encoding="latin-1") as file:
    content1 = file.read()

with open(file_path2, encoding="latin-1") as file:
    content2 = file.read()

with open(file_path3, encoding="latin-1") as file:
    content3 = file.read()

with open(file_path4, encoding="latin-1") as file:
    content4 = file.read()

with open(file_path5, encoding="latin-1") as file:
    content5 = file.read()

with open(file_path6, encoding="latin-1") as file:
    content6 = file.read()

with open(file_path7, encoding="latin-1") as file:
    content7 = file.read()



In [ ]:
# Sample text
text_obama = content1 + content2 + content3 + content4 + content5 + content6 + content7

A_obama, O_obama, num_to_word_obama, num_to_pos_obama = supervisedHMM(text_obama)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


In [ ]:
top_trump = generate_top(A_trump, O_trump, num_to_word_trump, num_to_pos_trump)
top_obama = generate_top(A_obama, O_obama, num_to_word_obama, num_to_pos_obama)

print(top_trump)

State $:
  Word: 'december', Probability: 1.0000

State CC:
  Word: 'elizabeth', Probability: 0.0016
  Word: 'or', Probability: 0.0482
  Word: 'but', Probability: 0.0700
  Word: 'and', Probability: 0.8802

State CD:
  Word: 'billion', Probability: 0.0150
  Word: '15', Probability: 0.0150
  Word: '3', Probability: 0.0180
  Word: '2', Probability: 0.0240
  Word: '10', Probability: 0.0240
  Word: 'four', Probability: 0.0269
  Word: 'three', Probability: 0.0389
  Word: 'two', Probability: 0.0509
  Word: 'million', Probability: 0.0599
  Word: 'one', Probability: 0.1647

State DT:
  Word: 'no', Probability: 0.0086
  Word: 'those', Probability: 0.0122
  Word: 'that', Probability: 0.0194
  Word: 'these', Probability: 0.0202
  Word: 'every', Probability: 0.0230
  Word: 'an', Probability: 0.0360
  Word: 'all', Probability: 0.0425
  Word: 'this', Probability: 0.0742
  Word: 'a', Probability: 0.1814
  Word: 'the', Probability: 0.5500

State EX:
  Word: 'matthew', Probability: 0.0500
  Word: 'ellie

In [ ]:
def gen_wordcloud(O, num_to_word):
  #for state_idx in range(len(O)):
  for state_idx in [8,9]:
    state_probs = O[state_idx]
    word_frequencies = {num_to_word[word_idx]: prob for word_idx, prob in enumerate(state_probs)}

    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_frequencies)
    plt.figure(figsize=(10, 5))
    plt.title(f"Word Cloud for State {state_idx}")
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
text_trump = re.sub(r'[\\$\\(\\)\\;\\:]','', text_trump)

sentences = re.split(r'[!?\.]\s*', text_trump)

text_obama = re.sub(r'[\\$\\(\\)\\;\\:]','', text_trump)

sentences_obama = re.split(r'[!?\.]\s*', text_obama)


print(sentences)

['Mr', 'Speaker, Mr', 'Vice President, Members of Congress, the First Lady of the United States, and my fellow Americans\nLess than 1 year has passed since I first stood at this podium, in this majestic chamber, to speak on behalf of the American PeopleÑand to address their concerns, their hopes, and their dreams', 'That night, our new Administration had already taken swift action', 'A new tide of optimism was already sweeping across our land', 'Each day since, we have gone forward with a clear vision and a righteous missionÑto make America great again for all Americans', 'Over the last year, we have made incredible progress and achieved extraordinary success', 'We have faced challenges we expected, and others we could never have imagined', 'We have shared in the heights of victory and the pains of hardship', 'We endured floods and fires and storms', 'But through it all, we have seen the beauty of AmericaÕs soul, and the steel in AmericaÕs spine', 'Each test has forged new American her

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Bidirectional

# Sample larger corpus
corpus = sentences_obama

# Tokenize words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Convert sentences to sequences
input_sequences = []
for sentence in corpus:
    tokens = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokens)):
        input_sequences.append(tokens[:i+1])

# Pad sequences
max_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding="pre")

# Split into input (X) and output (y)
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

# Load Pretrained GloVe Embeddings
glove_path = "/content/drive/My Drive/glove.6B.50d.txt"  # Update the path if needed

# Step 3: Load GloVe embeddings into a dictionary
embedding_dim = 50  # Change this based on the GloVe file you're using (e.g., 50d, 100d, 200d)
embedding_index = {}

with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]  # First element is the word
        coef = np.asarray(values[1:], dtype="float32")  # Remaining elements are the vector
        embedding_index[word] = coef

# Create Embedding Matrix
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    vector = embedding_index.get(word)
    if vector is not None:
        embedding_matrix[i] = vector

# Define the model
model = Sequential([
    Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length - 1, trainable=False),
    Bidirectional(LSTM(150, return_sequences=True)),
    Dropout(0.2),
    LSTM(150),
    Dropout(0.2),
    Dense(100, activation="relu"),
    Dense(total_words, activation="softmax")
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X, y, epochs=100, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 195s 383ms/step - accuracy: 0.0372 - loss: 7.1394
Epoch 2/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 213s 407ms/step - accuracy: 0.0477 - loss: 6.6060
Epoch 3/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 188s 388ms/step - accuracy: 0.0617 - loss: 6.4636
Epoch 4/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 203s 390ms/step - accuracy: 0.0714 - loss: 6.2758
Epoch 5/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 199s 385ms/step - accuracy: 0.0771 - loss: 6.0655
Epoch 6/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 203s 387ms/step - accuracy: 0.0964 - loss: 5.8406
Epoch 7/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 188s 387ms/step - accuracy: 0.1037 - loss: 5.5954
Epoch 8/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 202s 386ms/step - accuracy: 0.1079 - loss: 5.4208
Epoch 9/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 201s 384ms/step - accuracy: 0.1180 - loss: 5.2371
Epoch 10/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 200s 378ms/step - accuracy: 0.1300 - loss: 5.0186
Epoch 11/100
486/486 ━━━━━━━━━━━━━━━━━━━━ 203s 381ms/step - accuracy: 0.1363 - loss: 4.83

In [ ]:
model.save('/content/drive/MyDrive/obamaRNN.keras')

In [ ]:
from nltk.corpus import wordnet
nltk.download('wordnet')

def get_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name().replace("_", " "))
    return list(synonyms)

# Example usage
print(get_synonyms("great"))

In [ ]:
import heapq

def generate_sentence(model, seed_text, next_words=10, beam_width=3):
    sequences = [(seed_text, 1.0)]  # (sentence, probability)

    for _ in range(next_words):
        all_candidates = []

        for sentence, prob in sequences:
            token_list = tokenizer.texts_to_sequences([sentence])[0]
            token_list = pad_sequences([token_list], maxlen=max_length - 1, padding="pre")

            predictions = model.predict(token_list)[0]
            top_words = np.argsort(predictions)[-beam_width:]  # Get top-k words

            for word_index in top_words:
                word = tokenizer.index_word.get(word_index, "")
                if word:
                    new_sentence = sentence + " " + word
                    new_prob = prob * predictions[word_index]  # Adjust probability
                    all_candidates.append((new_sentence, new_prob))

        # Keep top-k sequences
        sequences = heapq.nlargest(beam_width, all_candidates, key=lambda x: x[1])

    return sequences[0][0]


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_word(model, input_sentence, max_length):
    # Split sentence into words
    words = input_sentence.split()

    # Find the index of the blank word "__"
    if "__" not in words:
        raise ValueError("The input sentence must contain a blank '__' to predict the missing word.")

    blank_index = words.index("__")

    # Convert words to sequences
    sequence = [tokenizer.word_index.get(word, 0) for word in words if word != "__"]

    # Pad sequence to match model input size
    padded_sequence = pad_sequences([sequence], maxlen=max_length - 1, padding='pre')

    # Predict the next word
    predicted_index = np.argmax(model.predict(padded_sequence), axis=-1)[0]

    # Convert index to word
    predicted_word = tokenizer.index_word.get(predicted_index, "UNKNOWN")

    # Replace "__" with the predicted word
    words[blank_index] = predicted_word

    # Return the completed sentence
    return " ".join(words)


In [ ]:
# !pip install py_thesaurus
# from py_thesaurus import Thesaurus

def generate_word_target(model, input_sentence, target_word, max_length):
    # Split sentence into words
    words = input_sentence.split()

    blank_index = words.index(target_word)

    # Convert words to sequences
    sequence = [tokenizer.word_index.get(word, 0) for word in words if word != target_word]

    # Pad sequence to match model input size
    padded_sequence = pad_sequences([sequence], maxlen=max_length - 1, padding='pre')

    predictions = model.predict(padded_sequence)[0]  # Get probability distribution

    # Get synonyms for the target word
    synonyms = get_synonyms(target_word)

    # Get probabilities for the synonyms
    synonym_probs = {
        synonym: predictions[tokenizer.word_index.get(synonym, 0)]
        for synonym in synonyms if synonym in tokenizer.word_index
    }

    sorted_synonyms = dict(sorted(synonym_probs.items(), key=lambda item: item[1], reverse=True))


    # Return the completed sentence
    return sorted_synonyms

In [ ]:
from tensorflow.keras.models import load_model

trumpRNN = load_model('/content/drive/MyDrive/trumpRNN.keras')
obamaRNN = load_model('/content/drive/MyDrive/obamaRNN.keras')

In [ ]:
# print(generate_sentence(trumpRNN, "my fellow", next_words=10))
# print(generate_word(obamaRNN, "god bless the people of __", 20))
print('obama', generate_word_target(obamaRNN, "i love this happy country", "happy", max_length))
print('trump', generate_word_target(trumpRNN, "i love this amazing country", "amazing", max_length))